In [1]:
import json
import nltk
import math
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
#stemming
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
 
#for word in input_str:
 #   print(lemmatizer.lemmatize(word))

In [2]:
#Python library for interacting with JIRA via REST APIs.
#https://jira.readthedocs.io/en/latest/
from jira import JIRA
#connect to the jira sever using HTTP basic auth
jira = JIRA(basic_auth=('bsse0914@iit.du.ac.bd', 'MbaYDePZRnDgAdL062r83605'), options={'server':'https://pg-req.atlassian.net'})
#The library is able to load the credentials from inside the ~/.netrc file, 
#so put them there instead of keeping them in your source code.
#need to work on it - how to handle authentication
#do i need this auth if i am already logged in?*

In [5]:
projects = jira.projects()
num_of_projects=len(projects)
issueCount=0
for i in range(0,num_of_projects):
    print(projects[i])
    #jql
    issues_in_proj = jira.search_issues('project='+str(projects[i]))
    print(len(issues_in_proj))
    issueCount+=len(issues_in_proj)
print(issueCount)

C1R
12
FAZ
5
FEP
4
RMS
0
SAAJ
7
28


In [6]:
import re
import string 
def remove_punctuation(text):
    no_punc = "".join([c for c in text if c not in string.punctuation])
    return no_punc.lower()

def tokenize(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(text)
    porter_stemmer= PorterStemmer()
    lemmatizer=WordNetLemmatizer()

    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    terms_list = []
    #stemmed_sentence = []

    #Stemming is a little more aggressive. It cuts off prefixes and/or endings of words
    #based on common ones. It can sometimes be helpful, but not always because often times 
    #the new word is so much a root that it loses its actual meaning. 
    #Lemmatizing, on the other hand, maps common words into one base. 
    #Unlike stemming though, it always still returns a proper word that can be found in the dictionary.
    for w in word_tokens:
        if w not in stop_words:
            #w=porter_stemmer.stem(w)
            w=lemmatizer.lemmatize(w)
            terms_list.append(w)

    #print(word_tokens)
    #print("terms list")
    #print(terms_list)
    
    return terms_list

In [7]:
cnt=0
assignee_terms_map={}
terms_assignee_map={}
terms_assignee_list=[]
account_id_map={}

for p in projects:
    pro_key='project='
    pro_key+=str(p.key)
    project_key=pro_key
    size = 100
    initial = 0
    issueCount=0
    while True:
        start= initial*size
        issues = jira.search_issues(project_key,  start,size)
        issueCount+=len(issues)
        if len(issues) == 0:
            break
        initial += 1
        for issue in issues:
            if issue.fields.assignee and issue.fields.assignee!=None:
                cnt+=1
                summary = issue.fields.summary
                des = issue.fields.description
                assignee = str(issue.fields.assignee)
                account_id = str(issue.fields.assignee.accountId)
                #dateofissue
                #issuetype
                #done #todo #inprogress
                #assignee - assignee,date (dict)
                
                des=remove_punctuation(des)
 
                bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
                des = ''.join(i for i in des if not i in bad_chars)
                tokenized_des=tokenize(des)
        
                summary=remove_punctuation(summary)
 
                bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
                summary = ''.join(i for i in summary if not i in bad_chars)
                tokenized_summary=tokenize(summary)
               
                #list of all terms existing in an issue
                dupli_terms=tokenized_des+tokenized_summary
            
                terms = list(dict.fromkeys(dupli_terms))
                #terms_dict=term:assignee.time
                
                keylist=[]
                for key,val in assignee_terms_map.items():
                    keylist.append(key)
                                
                if assignee in keylist:
                    assignee_terms_map[assignee]=assignee_terms_map[assignee]+terms
                else:
                    assignee_terms_map[assignee]=terms
                    #assignee_terms_map[assignee.dev_name]=terms_dict

                    account_id_map[assignee]=account_id
    
                terms_assignee_list.append(assignee)
                #append assignee date
                #assignee is a dict -dev,date
            
                for term in terms:
                    if term in terms_assignee_map:
                        terms_assignee_map[term]=terms_assignee_map[term]+terms_assignee_list
                    else:
                        terms_assignee_map[term]=terms_assignee_list  
                    
                    #terms_assignee_date_map
                    
                terms_assignee_list=[]

print("Final map:")
print(assignee_terms_map)

print("Account ID map:")
print(account_id_map)

Final map:
{'Moumita Asad': ['write', 'icse', 'paper', 'including', 'finding', 'regression', 'analysis', 'primary', 'demonstration', 'write', 'blog', 'productivity', 'employee', 'wfh', 'publish', 'dsse', 'website', 'explore', 'literature', 'find', 'paper', 'related', 'employee', 'productivity', 'work', 'home'], 'Noshin Tahsin': ['regression', 'analysis', 'data', 'write', 'paper', 'finding', 'far', 'arxiv', 'create', 'pie', 'chart', 'data', 'visualization', 'clean', 'preprocess', 'data', 'change', 'column', 'name', 'drop', 'invalid', 'encode', 'categorical', 'find', 'productivityrelated', 'factor', 'related', 'paper', 'prepare', 'survey', 'questionnare', 'getting', 'list', 'view', 'queried', 'result', 'user', 'vscode', 'extension', 'exploring', 'feature', 'better', 'strategy', 'viewing', 'implementing', 'search', 'viewer', '404', 'error', 'need', 'solve', 'khay', 'oneksolve', 'kor', 'assignee', 'suggestion', 'table', 'add', 'image', 'along', 'name', 'make', 'ui', 'attractive', 'change',

In [11]:
#for i in range(0,num_of_projects):
 #   all_issues=jira.search_issues('project='+str(projects[i]))
  #  num_of_issues=len(all_issues)
   # for j in range(0,num_of_issues):
        #if i!=0 or j!=14:
        #issue = data["projects"][i]['issues'][j]
     #   issue = all_issues[j]   
    #    
#print(assignee_terms_map)
#print(terms_assignee_map)
    

In [12]:
print(len(assignee_terms_map))
available_devlist=[]

for key,val in assignee_terms_map.items():
    available_devlist.append(key)
print(available_devlist) 
#del assignee_terms_map["None"]
for key,val in  assignee_terms_map.items():
    print(key, "=>", val)
    #print(key)
    #print(len(val))   

5
['Moumita Asad', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Toukir Ahammed', 'A. T. M. Fazlay Rabbi']
Moumita Asad => ['write', 'icse', 'paper', 'including', 'finding', 'regression', 'analysis', 'primary', 'demonstration', 'write', 'blog', 'productivity', 'employee', 'wfh', 'publish', 'dsse', 'website', 'explore', 'literature', 'find', 'paper', 'related', 'employee', 'productivity', 'work', 'home']
Noshin Tahsin => ['regression', 'analysis', 'data', 'write', 'paper', 'finding', 'far', 'arxiv', 'create', 'pie', 'chart', 'data', 'visualization', 'clean', 'preprocess', 'data', 'change', 'column', 'name', 'drop', 'invalid', 'encode', 'categorical', 'find', 'productivityrelated', 'factor', 'related', 'paper', 'prepare', 'survey', 'questionnare', 'getting', 'list', 'view', 'queried', 'result', 'user', 'vscode', 'extension', 'exploring', 'feature', 'better', 'strategy', 'viewing', 'implementing', 'search', 'viewer', '404', 'error', 'need', 'solve', 'khay', 'oneksolve', 'kor', 'assignee', 's

In [13]:
#now i have mapped the assignee ans terms
#now need to map term => dev
print(len(terms_assignee_map))
add=0
#for key,val in  terms_assignee_map.items():
    #print(key, "=>", val)
    #print(len(val))
    #add=add+len(val)
    #print(add)
    
#1347 unique terms??

#did nothing for sparse terms - that occur less frequently
#nltk.download('wordnet')
#print(len(terms_assignee_map['paper']))
#for i in range (0,len(terms_assignee_map['paper'])):
    #print(terms_assignee_map['paper'][i])
print(terms_assignee_map)

139
{'write': ['Moumita Asad', 'Noshin Tahsin', 'Moumita Asad', 'Mridha Md. Nafis Fuad'], 'icse': ['Moumita Asad'], 'paper': ['Moumita Asad', 'Noshin Tahsin', 'Noshin Tahsin', 'Moumita Asad', 'Mridha Md. Nafis Fuad'], 'including': ['Moumita Asad'], 'finding': ['Moumita Asad', 'Noshin Tahsin'], 'regression': ['Moumita Asad', 'Noshin Tahsin'], 'analysis': ['Moumita Asad', 'Noshin Tahsin'], 'data': ['Noshin Tahsin', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Noshin Tahsin', 'Toukir Ahammed'], 'far': ['Noshin Tahsin'], 'arxiv': ['Noshin Tahsin'], 'primary': ['Moumita Asad'], 'demonstration': ['Moumita Asad'], 'blog': ['Moumita Asad'], 'productivity': ['Moumita Asad', 'Moumita Asad'], 'employee': ['Moumita Asad', 'Moumita Asad'], 'wfh': ['Moumita Asad'], 'publish': ['Moumita Asad'], 'dsse': ['Moumita Asad'], 'website': ['Moumita Asad'], 'create': ['Noshin Tahsin', 'Mridha Md. Nafis Fuad'], 'pie': ['Noshin Tahsin'], 'chart': ['Noshin Tahsin', 'Mridha Md. Nafis Fuad'], 'visualization': ['Nosh

In [14]:
#tokenizing the new issue
#new_des=data["projects"][0]['issues'][14]['description']

#all_new_issues=jira.search_issues('project='+str(projects[0]))
#new_issue=all_new_issues[0]

new_issue=jira.issue('C1R-12')
new_des=new_issue.fields.description

new_des=remove_punctuation(new_des)
bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
new_des = ''.join(i for i in new_des if not i in bad_chars)
tokenized_new_des=tokenize(new_des)

#new_summary=data["projects"][0]['issues'][14]['description']
#new_date

new_summary=new_issue.fields.summary

new_summary=remove_punctuation(new_summary)
bad_chars=['!','@', '#', '$','%', '^', '&','*','(',')','-','+']
new_summary = ''.join(i for i in new_summary if not i in bad_chars)
tokenized_new_summary=tokenize(new_summary)

new_terms=tokenized_new_des+tokenized_new_summary
print(new_terms)

#now i have the new issue terms

['explore', 'literature', 'find', 'tool', 'paper', 'icse', 'related', 'regression', 'analysis', 'related', 'data', 'find', 'tool', 'paper', 'icse']


In [15]:
#now calculate score for developer suggestion
#print(len(new_terms))
#term - dev_dict{dev: ,date:}
newterm_dev_map={}
#def getDevTermUsageInfo():
cnt=0
for newterm in new_terms:
    if newterm in terms_assignee_map:
        #print(cnt)
        cnt=cnt+1
        print(newterm)
        print(terms_assignee_map[newterm])
        newterm_dev_map[newterm]=terms_assignee_map[newterm]
        #returns a devlist for a single newterm
        #developers who has previously worked on that term
        #one dev name can occur multiple times
        #which implies he has worked on that term multiple times-more experienced!

#print(newterm_dev_map['everything'])
    

explore
['Moumita Asad']
literature
['Moumita Asad']
find
['Noshin Tahsin', 'Moumita Asad']
tool
['Mridha Md. Nafis Fuad']
paper
['Moumita Asad', 'Noshin Tahsin', 'Noshin Tahsin', 'Moumita Asad', 'Mridha Md. Nafis Fuad']
icse
['Moumita Asad']
related
['Noshin Tahsin', 'Moumita Asad']
regression
['Moumita Asad', 'Noshin Tahsin']
analysis
['Moumita Asad', 'Noshin Tahsin']
related
['Noshin Tahsin', 'Moumita Asad']
data
['Noshin Tahsin', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Noshin Tahsin', 'Toukir Ahammed']
find
['Noshin Tahsin', 'Moumita Asad']
tool
['Mridha Md. Nafis Fuad']
paper
['Moumita Asad', 'Noshin Tahsin', 'Noshin Tahsin', 'Moumita Asad', 'Mridha Md. Nafis Fuad']
icse
['Moumita Asad']


In [16]:
dev_termInfo={}

for dev in available_devlist:
    #print(dev)
    term_info={}
    #check if he has used a newterm and get the newterm
    for key in newterm_dev_map:
        if dev in newterm_dev_map[key]:
            #now i need the key-key is the newterm
            #calculate term info for the key
            #just need the fixfrequency
            #how many times dev is in newterm_dev_map[key]
            devnames=newterm_dev_map[key]
            #print(devnames)
            dev_count=devnames.count(dev)
            #print(key)
            #print(dev_count)
            #List-devcount,termusedate
            term_info[key]=dev_count #add term_use_date : last fixing time of a term by a developer
            
    dev_termInfo[dev]=term_info

    

In [17]:
for key,value in dev_termInfo.items():
    print(key," => ",value)
#Toukir Ahammed  =>  {'data': 2, (date last worked)}
#now i have the complex map of dev term term-info

Moumita Asad  =>  {'explore': 1, 'literature': 1, 'find': 1, 'paper': 2, 'icse': 1, 'related': 1, 'regression': 1, 'analysis': 1}
Noshin Tahsin  =>  {'find': 1, 'paper': 2, 'related': 1, 'regression': 1, 'analysis': 1, 'data': 3}
Mridha Md. Nafis Fuad  =>  {'tool': 1, 'paper': 1, 'data': 1}
Toukir Ahammed  =>  {'data': 1}
A. T. M. Fazlay Rabbi  =>  {}


In [18]:
prediction_list=[]

def devFixFreq(term):
    dev_use_count=0
    for dev in assignee_terms_map:
        terms_used=assignee_terms_map[dev]
        if term in terms_used:#if term in terms used keylist
            dev_use_count=dev_use_count+1
    return dev_use_count
    
dev_score={} 
for key in dev_termInfo:
    expertise=0
    termInfo=dev_termInfo[key]
    #print(termInfo)
    for info_term in termInfo:
        term_freq=termInfo[info_term]
        #print(info_term)
        #print(term_freq)
        ##dev=number of devs in the project 
        #len(available_devlist)
        
        #devfixfreq() : returns numof devs who fixed the term related bugs
        num_of_devs=len(available_devlist)
        tfIdf=term_freq*math.log(num_of_devs/devFixFreq(info_term))
        #What about time data?
        expertise+=tfIdf
    #end of for
    dev_score[key]=expertise
#end of for
print(dev_score)
    

{'Moumita Asad': 9.515127912330904, 'Noshin Tahsin': 6.219291046326574, 'Mridha Md. Nafis Fuad': 2.6310891599660815, 'Toukir Ahammed': 0.5108256237659907, 'A. T. M. Fazlay Rabbi': 0}


In [19]:
sorted_dev_score = sorted(dev_score.items() , reverse=True, key=lambda x: x[1])
# Iterate over the sorted sequence
for elem in sorted_dev_score :
    print(elem[0] , " ::" , elem[1] )

Moumita Asad  :: 9.515127912330904
Noshin Tahsin  :: 6.219291046326574
Mridha Md. Nafis Fuad  :: 2.6310891599660815
Toukir Ahammed  :: 0.5108256237659907
A. T. M. Fazlay Rabbi  :: 0


In [20]:
a=[]
a_id=[]
for i in range (0,len(sorted_dev_score)):
    a.append(sorted_dev_score[i][0])
    a_id.append(account_id_map[sorted_dev_score[i][0]])
    print(sorted_dev_score[i][0])
    print("Account ID")
    print(account_id_map[sorted_dev_score[i][0]])
    print("\n")

print(a)
print(a_id)

Moumita Asad
Account ID
5f5764e4bea5be0068101b9a


Noshin Tahsin
Account ID
5e299526bf04010e70c42927


Mridha Md. Nafis Fuad
Account ID
5ae1622f027e7a2ebfcc12d1


Toukir Ahammed
Account ID
5ca99ba2e929a772795e4ac9


A. T. M. Fazlay Rabbi
Account ID
5e9412627bc0680c2ca4dd39


['Moumita Asad', 'Noshin Tahsin', 'Mridha Md. Nafis Fuad', 'Toukir Ahammed', 'A. T. M. Fazlay Rabbi']
['5f5764e4bea5be0068101b9a', '5e299526bf04010e70c42927', '5ae1622f027e7a2ebfcc12d1', '5ca99ba2e929a772795e4ac9', '5e9412627bc0680c2ca4dd39']
